Theanoを使った計算を試してみる

In [1]:
#coding: utf-8
import theano
import theano.tensor as T

# シンボルの生成
# xはdoubleのスカラー型
x = T.dscalar('x')
print type(x)

# シンボルを組み立てて数式を定義（これもまたシンボル）
y = x ** 2
print type(y)

# シンボルを使って関数を定義
# ここでコンパイルされる
f = theano.function(inputs=[x], outputs=y)
print type(f)

# 関数を使ってxに具体的な値を入れてyを計算
print f(1)
print f(2)
print f(3)
print f(4)
print f(5)

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 4007)


<class 'theano.tensor.var.TensorVariable'>
<class 'theano.tensor.var.TensorVariable'>
<class 'theano.compile.function_module.Function'>
1.0
4.0
9.0
16.0
25.0


In [2]:
#coding: utf-8
import theano
import theano.tensor as T

# シンボルの生成
# xはdoubleのスカラー型
x = T.vector('x')
print type(x)

# シンボルを組み立てて数式を定義（これもまたシンボル）
y = x ** 2
print type(y)

# シンボルを使って関数を定義
# ここでコンパイルされる
f = theano.function(inputs=[x], outputs=y)
print type(f)

# 関数を使ってxに具体的な値を入れてyを計算
print f([1,2,3])


<class 'theano.tensor.var.TensorVariable'>
<class 'theano.tensor.var.TensorVariable'>
<class 'theano.compile.function_module.Function'>
[ 1.  4.  9.]


In [3]:
#coding: utf-8
import theano
import theano.tensor as T

# シンボルの生成
# xはdoubleのスカラー型
x = T.dmatrix('x')
print type(x)

# シンボルを組み立てて数式を定義（これもまたシンボル）
y = x ** 3
print type(y)

# シンボルを使って関数を定義
# ここでコンパイルされる
f = theano.function(inputs=[x], outputs=y)
print type(f)

# 関数を使ってxに具体的な値を入れてyを計算
print f([[1,2,3], [4,5,6]])


<class 'theano.tensor.var.TensorVariable'>
<class 'theano.tensor.var.TensorVariable'>
<class 'theano.compile.function_module.Function'>
[[   1.    8.   27.]
 [  64.  125.  216.]]


シグモイド関数を使った計算をしてみる

In [4]:
#coding: utf-8
import theano
import theano.tensor as T

# シンボルを定義
x = T.dmatrix('x')

# シンボルを組み合わせて数式を定義
# s = T.nnet.sigmoid(x)でもOK
s = 1 / (1 + T.exp(-x))

# シンボルを使って関数化
sigmoid = theano.function(inputs=[x], outputs=s)

# 実際の値を使って計算
print sigmoid([[0, 1], [-1, -2]])

[[ 0.5         0.73105858]
 [ 0.26894142  0.11920292]]


共有変数について学ぶ  
GPUのメモリに乗せるデータはfloat型（theano.config.floatX）にする必要がある。

In [5]:
#coding: utf-8
import numpy as np
import theano

# データを共有変数に読み込む
data = np.array([[1,2,3], [4,5,6]], dtype=theano.config.floatX)
X = theano.shared(data, name='X', borrow=True)
print type(X)
print X.get_value()

<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>
[[ 1.  2.  3.]
 [ 4.  5.  6.]]


線形回帰モデルの例

In [6]:
#coding: utf-8
import numpy as np
import theano
import theano.tensor as T

# 共有変数の定義
# 具体的な数値で初期化
W = theano.shared(np.array([[1,2,3], [4,5,6]], dtype=theano.config.floatX),
                  name='W',
                  borrow=True)
b = theano.shared(np.array([1, 1], dtype=theano.config.floatX),
                  name='b',
                  borrow=True)

# 共有変数の値の取得
print W.get_value()
print b.get_value()

# シンボルの生成
x = T.vector('x')

# シンボルと共有変数を組み立てて数式を定義
y = T.dot(W, x) + b
print type(y)

# 関数を定義
f = theano.function([x], y)

# 関数の使用
print f([1, 1, 1])

[[ 1.  2.  3.]
 [ 4.  5.  6.]]
[ 1.  1.]
<class 'theano.tensor.var.TensorVariable'>
[  7.  16.]


自動微分の例1  

In [7]:
#coding: utf-8
import theano
import theano.tensor as T

x = T.dscalar('x')

# 微分される数式のシンボルを定義
y = x ** 2

# yをxに関して微分
# y'=2xになる
# costは微分する数式のシンボル
# wrtには微分する変数のシンボル
gy = T.grad(cost=y, wrt=x)

# 微分係数を求める関数を定義
f = theano.function(inputs=[x], outputs=gy)
print theano.pp(f.maker.fgraph.outputs[0])

# 具体的なxを与えて微分係数を求める
print f(2)
print f(3)
print f(4)

(TensorConstant{2.0} * x)
4.0
6.0
8.0


自動微分の例2

In [8]:
#coding: utf-8
import theano
import theano.tensor as T

x = T.dscalar('x')

# 微分される数式のシンボルを定義
y = T.exp(x)

# yをxに関して微分
gy = T.grad(cost=y, wrt=x)

# 微分係数を求める関数を定義
f = theano.function(inputs=[x], outputs=gy)
print theano.pp(f.maker.fgraph.outputs[0])

# 具体的なxを与えて微分係数を求める
print f(2)
print f(3)
print f(4)

exp(x)
7.38905609893
20.0855369232
54.5981500331


自動微分の例3

In [9]:
import theano
import theano.tensor as T
import numpy as np

x = T.dscalar('x')

# 微分される数式のシンボルを定義
y = T.sin(x)

# yをxに関して微分
gy = T.grad(cost=y, wrt=x)

# 微分係数を求める関数を定義
f = theano.function(inputs=[x], outputs=gy)
print theano.pp(f.maker.fgraph.outputs[0])

# 具体的なxを与えて微分係数を求める
print f(0)
print f(np.pi / 2)
print f(np.pi)

cos(x)
1.0
6.12323399574e-17
-1.0


自動微分の例4

In [10]:
#coding: utf-8
import theano
import theano.tensor as T
import numpy as np

x = T.dscalar('x')

# 微分される数式のシンボルを定義
y = (x - 4) * (x ** 2 + 6)

# yをxに関して微分
gy = T.grad(cost=y, wrt=x)

# 微分係数を求める関数を定義
f = theano.function(inputs=[x], outputs=gy)
print theano.pp(f.maker.fgraph.outputs[0])

# 具体的なxを与えて微分係数を求める
print f(0)
print f(1)
print f(2)

Elemwise{Composite{(i0 + sqr(i1) + (i2 * (i3 + i1) * i1))}}(TensorConstant{6.0}, x, TensorConstant{2.0}, TensorConstant{-4.0})
6.0
1.0
2.0


2クラスロジスティック回帰の実装

In [3]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
# Wは784このベクトル
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
# bはスカラー
b = theano.shared(0., name="b")

print("Initial model:")
print "w.get_value()= "+str(w.get_value())
print "b.get_value()= "+str(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Initial model:
w.get_value()= [ -1.40901876e+00  -4.06532210e-02   1.73266515e+00  -9.74324657e-02
  -4.00310135e-01  -4.51539258e-01  -2.22380678e+00   1.14709569e+00
   1.16744138e+00   4.69016241e-01   9.79367463e-01  -1.33420583e+00
  -1.47530410e-01   4.29685320e-02  -1.42157032e+00  -3.00250041e-01
   1.74507191e+00  -2.49363009e+00  -7.97944078e-01   1.32003104e+00
   1.30738894e+00  -5.10907151e-01  -1.66001313e+00   2.43814481e-01
   1.22394615e+00  -1.48362324e+00   4.07245921e-02  -3.16789848e-01
   1.30910993e-01   1.23773349e+00  -1.29887588e-01   1.45137562e-01
   3.58722636e-01   1.61506770e+00  -9.93656649e-01  -1.51081384e+00
   1.36781751e+00   1.59008457e+00  -1.56367902e-01   1.83172790e-01
  -8.04269506e-01  -7.28243613e-01   1.13604705e-01  -1.21277799e+00
   1.46622651e-01   7.14985587e-01  -6.01820830e-03   6.11225982e-01
  -5.53480986e-01   9.44136354e-01   1.21137257e+00  -1.17259296e+00
  -1.78245561e-01   9.28702498e-01  -8.50860990e-01  -1.95018698e-01
  -7